In [1]:
from getEPH import *
import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

/home/pipe/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data1 = pd.read_csv('data/cleanDatat310.csv')
data2 = categorize(data1)
data3 = schoolYears(data2)


In [3]:
data3.columns

Index([u'CODUSU', u'NRO_HOGAR', u'COMPONENTE', u'AGLOMERADO', u'PONDERA',
       u'familyRelation', u'female', u'age', u'schoolLevel', u'finishedYear',
       u'lastYear', u'activity', u'empCond', u'unempCond', u'ITF', u'IPCF',
       u'P47T', u'P21', u'primary', u'secondary', u'university'],
      dtype='object')

In [20]:
cleanData = data3.copy()
cleanData.drop_duplicates(inplace=True)
cleanData['education'] = cleanData.primary + cleanData.secondary + cleanData.university
cleanData['education2'] = cleanData['education'] ** 2
cleanData['age2'] = cleanData['age'] ** 2
#ME QUEDO CON LOS QUE TRABAJAN Y TIENEN INGRESOS, SOLO USAMOS ESTOS
jobsAndIncome = (cleanData.activity == 1) & (cleanData.P21 > 0) & ((cleanData.familyRelation == 1)|(cleanData.familyRelation == 2))
cleanData['id'] = (cleanData.CODUSU.astype(str) + cleanData.NRO_HOGAR.astype(str))
cleanData = cleanData.copy().loc[jobsAndIncome,
                              ['id',
                            'familyRelation',
                              'age',
                              'age2',
                              'female',
                              'education',
                              'education2']]


In [21]:
cleanData.education.isnull().sum()

21

In [22]:
probando = cleanData.loc[13:28]
probando

,id,familyRelation,age,age2,female,education,education2
13,3113561,1,46,2116,0,12.0,144.0
14,3113561,2,48,2304,1,7.0,49.0
17,1563001,1,49,2401,0,17.0,289.0
22,2554811,2,65,4225,1,17.0,289.0
28,2858412,1,45,2025,1,7.0,49.0


In [23]:
pivot = probando.pivot(index='id', columns='familyRelation')
pivot

age          age2         female      education        \
familyRelation     1     2       1       2      1    2         1     2   
id                                                                       
1563001         49.0   NaN  2401.0     NaN    0.0  NaN      17.0   NaN   
2554811          NaN  65.0     NaN  4225.0    NaN  1.0       NaN  17.0   
2858412         45.0   NaN  2025.0     NaN    1.0  NaN       7.0   NaN   
3113561         46.0  48.0  2116.0  2304.0    0.0  1.0      12.0   7.0   

               education2         
familyRelation          1      2  
id                                
1563001             289.0    NaN  
2554811               NaN  289.0  
2858412              49.0    NaN  
3113561             144.0   49.0

In [25]:
np.where(np.isnan(pivot.values),0,pivot.values)


array([[  4.90000000e+01,   0.00000000e+00,   2.40100000e+03,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.70000000e+01,   0.00000000e+00,   2.89000000e+02,
          0.00000000e+00],
       [  0.00000000e+00,   6.50000000e+01,   0.00000000e+00,
          4.22500000e+03,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   1.70000000e+01,   0.00000000e+00,
          2.89000000e+02],
       [  4.50000000e+01,   0.00000000e+00,   2.02500000e+03,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          7.00000000e+00,   0.00000000e+00,   4.90000000e+01,
          0.00000000e+00],
       [  4.60000000e+01,   4.80000000e+01,   2.11600000e+03,
          2.30400000e+03,   0.00000000e+00,   1.00000000e+00,
          1.20000000e+01,   7.00000000e+00,   1.44000000e+02,
          4.90000000e+01]])

In [24]:
pivot.shape

(4, 10)